## titanic dataset

In [17]:
#scikit-learn
import sklearn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
dataset = pd.read_excel('titanic3.xls')
dataset.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [19]:
dataset.drop(columns=['name', 'ticket','cabin','boat','body','home.dest'],inplace=True)
dataset.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,S
1,1,1,male,0.9167,1,2,151.5500,S
2,1,0,female,2.0000,1,2,151.5500,S
3,1,0,male,30.0000,1,2,151.5500,S
4,1,0,female,25.0000,1,2,151.5500,S


In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   int64  
 5   parch     1309 non-null   int64  
 6   fare      1308 non-null   float64
 7   embarked  1307 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 81.9+ KB


In [21]:
dataset.dropna(inplace=True)

In [22]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1043 non-null   int64  
 1   survived  1043 non-null   int64  
 2   sex       1043 non-null   object 
 3   age       1043 non-null   float64
 4   sibsp     1043 non-null   int64  
 5   parch     1043 non-null   int64  
 6   fare      1043 non-null   float64
 7   embarked  1043 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 73.3+ KB


In [23]:
dataset_dummies = pd.get_dummies(dataset)
dataset_dummies.head()

,pclass,survived,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,1,1,29.0000,0,0,211.3375,1,0,0,0,1
1,1,1,0.9167,1,2,151.5500,0,1,0,0,1
2,1,0,2.0000,1,2,151.5500,1,0,0,0,1
3,1,0,30.0000,1,2,151.5500,0,1,0,0,1
4,1,0,25.0000,1,2,151.5500,1,0,0,0,1


In [24]:
#podział  na zbiór testowy i treningowy
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 834
test_dataset length: 209


In [25]:
train_stats = train_dataset.describe()
train_stats.pop('survived')
train_stats = train_stats.transpose()

In [26]:
train_labels = train_dataset.pop('survived')
test_labels = test_dataset.pop('survived')

In [29]:
#standaryzacja
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [30]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [32]:
normed_train_data.head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
807,0.943720,-1.464814,1.573227,1.885220,-0.035832,1.292699,-1.292699,-0.501946,-0.227245,0.577927
453,-0.259739,-0.612818,-0.539609,-0.511538,0.648004,-0.772648,0.772648,-0.501946,-0.227245,0.577927
621,0.943720,-0.754818,0.516809,-0.511538,-0.499369,1.292699,-1.292699,-0.501946,-0.227245,0.577927
713,0.943720,-0.612818,-0.539609,-0.511538,-0.501482,-0.772648,0.772648,-0.501946,4.395256,-1.728249
834,0.943720,0.168177,-0.539609,-0.511538,-0.490485,-0.772648,0.772648,-0.501946,-0.227245,0.577927


In [33]:
from sklearn.neural_network import MLPClassifier

In [43]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(10, ), 
                    max_iter=1000, 
                    activation = 'relu',
                    alpha=0.0001,
                    solver='lbfgs', 
                    verbose=True)

In [44]:
mlp_clf.fit(normed_train_data, train_labels);

In [45]:
y_pred = mlp_clf.predict(normed_test_data)

In [46]:
pred_results = pd.DataFrame({'y_test': test_labels,
                             'y_pred': y_pred})

pred_results.sample(10)

,y_test,y_pred
346,0,0
208,1,1
413,0,0
694,0,0
316,0,0
1089,0,0
24,1,1
722,0,0
337,1,1
144,1,1


In [48]:
accuracy_score(test_labels, y_pred)

0.7942583732057417

In [51]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from  sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [52]:
print('Kappa:',cohen_kappa_score(test_labels, y_pred))
print('Accuracy:', accuracy_score(test_labels, y_pred))
print('Recall:',recall_score(test_labels, y_pred))
print('Precison:',precision_score(test_labels, y_pred))
print('F1 score:',f1_score(test_labels, y_pred))
print('Macierz pomyłek:')
print(confusion_matrix(test_labels, y_pred))
print(classification_report(test_labels, y_pred))

Kappa: 0.5591366200637724
Accuracy: 0.7942583732057417
Recall: 0.6547619047619048
Precison: 0.7971014492753623
F1 score: 0.7189542483660131
Macierz pomyłek:
[[111  14]
 [ 29  55]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       125
           1       0.80      0.65      0.72        84

    accuracy                           0.79       209
   macro avg       0.79      0.77      0.78       209
weighted avg       0.79      0.79      0.79       209



In [53]:
#keras
#red wine quality
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

pd.options.display.float_format = '{:.6f}'.format
tf.__version__

'2.3.0'

In [57]:
#budowa modelu
def build_model():
    model = Sequential()
    model.add(Dense(64, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=2, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [58]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                704       
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1040      
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                

In [60]:
history = model.fit(normed_train_data, train_labels.values, epochs=20, validation_split=0.2)

Epoch 1/20
21/21 [==============================] - 0s 5ms/step - loss: 0.5954 - accuracy: 0.7841 - val_loss: 0.5872 - val_accuracy: 0.7904
Epoch 2/20
21/21 [==============================] - 0s 5ms/step - loss: 0.6035 - accuracy: 0.8111 - val_loss: 0.5825 - val_accuracy: 0.7784
Epoch 3/20
21/21 [==============================] - 0s 3ms/step - loss: 0.5870 - accuracy: 0.8066 - val_loss: 0.5757 - val_accuracy: 0.7844
Epoch 4/20
21/21 [==============================] - 0s 3ms/step - loss: 0.5968 - accuracy: 0.8111 - val_loss: 0.5744 - val_accuracy: 0.7904
Epoch 5/20
21/21 [==============================] - 0s 3ms/step - loss: 0.5826 - accuracy: 0.7976 - val_loss: 0.5662 - val_accuracy: 0.7844
Epoch 6/20
21/21 [==============================] - 0s 4ms/step - loss: 0.5662 - accuracy: 0.8096 - val_loss: 0.5650 - val_accuracy: 0.7784
Epoch 7/20
21/21 [==============================] - 0s 3ms/step - loss: 0.5718 - accuracy: 0.8066 - val_loss: 0.5572 - val_accuracy: 0.7964
Epoch 8/20
21/21 [==

In [61]:
y_pred = np.argmax(model.predict(normed_test_data), axis=-1)

In [62]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = y_pred
pred.head()

,survived,predictions
11,1,1
20,1,0
24,1,1
26,1,1
29,1,0


In [63]:
print('Kappa:',cohen_kappa_score(test_labels, y_pred))
print('Accuracy:', accuracy_score(test_labels, y_pred))
print('Recall:',recall_score(test_labels, y_pred))
print('Precison:',precision_score(test_labels, y_pred))
print('F1 score:',f1_score(test_labels, y_pred))
print('Macierz pomyłek:')
print(confusion_matrix(test_labels, y_pred))
print(classification_report(test_labels, y_pred))

Kappa: 0.5829967395007056
Accuracy: 0.8038277511961722
Recall: 0.6904761904761905
Precison: 0.7945205479452054
F1 score: 0.7388535031847133
Macierz pomyłek:
[[110  15]
 [ 26  58]]
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       125
           1       0.79      0.69      0.74        84

    accuracy                           0.80       209
   macro avg       0.80      0.79      0.79       209
weighted avg       0.80      0.80      0.80       209

